# Custom Serializers using Annotated Types

We saw earlier how to customize serializers by attaching the serializer to individual fields using the `@field_serializer` decorator.

With validators, we can either attach the validator to a field (or multiple fields) using a decorator, or we can attache the validator to an annotated type instead.

The same is true with serializers. We saw how we could attach a custom serializer to a field using a decorator, but we can also attach a custom serializer to an annotated type instead.

Suppose we want a datetime type that:
- can deserialize other datetime strings (such as `1/1/2020 3pm`)
- always stores the datetime as an aware UTC datetime
- serializes the datetime to a datetime object when serializing to a Python dict
- serializes to JSON using this format: `YYYY/MM/DD HH:MM AM/PM (UTC)`

We've seen how to all this using either decorators or anotated types. Let's start with the validators first, that we'll attach to an annotated type.

Nothing new here, we've seen this before.

In [1]:
from datetime import datetime
from typing import Annotated, Any

import pytz
from dateutil.parser import parse

from pydantic import BaseModel, AfterValidator, BeforeValidator


def make_utc(dt: datetime) -> datetime:
    if dt.tzinfo is None:
        dt = pytz.utc.localize(dt)
    else:
        dt = dt.astimezone(pytz.utc)
    return dt
    
def parse_datetime(value: Any):
    if isinstance(value, str):
        try:
            return parse(value)
        except Exception as ex:
            raise ValueError(str(ex))
    return value


DateTimeUTC = Annotated[datetime, BeforeValidator(parse_datetime), AfterValidator(make_utc)]

And we can use this annotated type in any model:

In [2]:
class Model(BaseModel):
    dt: DateTimeUTC

However, we also need to add a custom serializer for JSON serialization.

Let's do it using a decorator first, as we learned previously.

First, we'll create a function to output the string formatted datetime object:

In [3]:
def dt_json_serializer(dt: datetime) -> str:
    return dt.strftime("%Y/%m/%d %I:%M %p UTC")

In [4]:
dt_json_serializer(datetime(2020,1,1,15,0,0))

'2020/01/01 03:00 PM UTC'

Next, we need to attach this to our field in the model.

Now, we only need this for JSON serialization, and only if the field is not None, so we'll define our custom serializer that way:

In [5]:
from pydantic import field_serializer

In [6]:
class Model(BaseModel):
    dt: DateTimeUTC

    @field_serializer("dt", when_used="json-unless-none")
    def serialize_dt_to_json(self, value: datetime) -> str:
        return dt_json_serializer(value)

Let's try out our model:

In [7]:
m = Model(dt="2020/1/1 3pm")
m

Model(dt=datetime.datetime(2020, 1, 1, 15, 0, tzinfo=<UTC>))

And let's serialize the model:

In [8]:
m.model_dump()

{'dt': datetime.datetime(2020, 1, 1, 15, 0, tzinfo=<UTC>)}

In [9]:
m.model_dump_json()

'{"dt":"2020/01/01 03:00 PM UTC"}'

Ok, so this works as we intended.

But, this custom serializer could be attached to the annotated type instead of to the specific dfield directly in the model.

To do so, we are going to use `PlainSerializer` from Pydantic.

A plain serializer essentially replaces Pydantic's serializer with our own custom one (which is what we were doing with the decorator approach).

In [10]:
from pydantic import PlainSerializer

In [11]:
DateTimeUTC = Annotated[
    datetime, 
    BeforeValidator(parse_datetime), 
    AfterValidator(make_utc),
    PlainSerializer(dt_json_serializer),
]

The only thing missing here, is **when** shoudl the serialize be used - in our case we want `json-unless-none`.

To add this to the serializer, we simply specify it as a named argument:

In [12]:
DateTimeUTC = Annotated[
    datetime, 
    BeforeValidator(parse_datetime), 
    AfterValidator(make_utc),
    PlainSerializer(dt_json_serializer, when_used="json-unless-none"),
]

And, now, we have a completely reuseable type that contains custom validators and a serializer.

In [13]:
class Model(BaseModel):
    dt: DateTimeUTC

And let's try it out:

In [14]:
m = Model(dt="2020/1/1 3pm")
m

Model(dt=datetime.datetime(2020, 1, 1, 15, 0, tzinfo=<UTC>))

In [15]:
m.model_dump()

{'dt': datetime.datetime(2020, 1, 1, 15, 0, tzinfo=<UTC>)}

In [16]:
m.model_dump_json()

'{"dt":"2020/01/01 03:00 PM UTC"}'